In [2]:
import ipyrad.analysis as ipa
import toyplot 

In [3]:
data = "phys_both_outfiles/phys_both.snps.hdf5"

In [4]:
imap = {
    "berm" : ["Bermuda_100","Bermuda_103", "Bermuda_15", 
              "Bermuda_19", "Bermuda_28", "Bermuda_30", 
              "Bermuda_54", "Bermuda_55", "Bermuda_63",
              "Bermuda_65", "Bermuda_67", "Bermuda_68",
              "Bermuda_69", "Bermuda_94", "Bermuda_8", 
              "Bermuda_80", "Bermuda_84", "Bermuda_85",
              "Bermuda_94"],
    "guam" : ["Guam_104460", "Guam_104461", "Guam_104464",
              "Guam_104465"],
    "s_afr" : ["South_Africa"],
    "texas" : ["8_TX2017_028", "82_TX2016_049", "6_TX2017_027", 
               "59_TX2016_048", "42_TX2016_046", "13_TX2016_043"]
}

minmap = {i: 0.5 for i in imap}

In [5]:
minmap

{'berm': 0.5, 'guam': 0.5, 's_afr': 0.5, 'texas': 0.5}

In [6]:
struct = ipa.structure(
    name="test",
    data = data,
    imap = imap,
    minmap = minmap,
    mincov = 0.9,)

Samples: 29
Sites before filtering: 101658
Filtered (indels): 0
Filtered (bi-allel): 3584
Filtered (mincov): 100625
Filtered (minmap): 95712
Filtered (subsample invariant): 8687
Filtered (minor allele frequency): 0
Filtered (combined): 101651
Sites after filtering: 713
Sites containing missing values: 645 (90.46%)
Missing values in SNP matrix: 1024 (4.95%)
SNPs (total): 713
SNPs (unlinked): 89


In [21]:
struct.mainparams.burnin = 5000
struct.mainparams.numreps = 10000
struct.run(nreps=10, kpop=[2, 3, 4, 5], auto=True)

[####################] 100% 0:02:16 | running 28 structure jobs 


In [22]:
etable = struct.get_evanno_table([2, 3, 4, 5])
etable

,Nreps,lnPK,lnPPK,deltaK,estLnProbMean,estLnProbStdev
2,10,0.00,0.00,0.000,-653.60,18.661
3,10,-1.67,34.11,1.184,-655.27,28.818
4,10,-35.78,72.38,0.437,-691.05,165.522
5,10,36.60,0.00,0.000,-654.45,46.176


In [23]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf79dd224c7874373a3f34d0418a5e238" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 K (N ancestral populations) 650 660 670 680 690 estLnProbMean 0.0 0.5 1.0 1.5 deltaK

In [38]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [31]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]


In [32]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t3c1f4c641e9a4ee98c18e0973d5fd89f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Bermuda_100 Bermuda_103 Bermuda_15 Bermuda_19 Bermuda_28 Bermuda_30 Bermuda_54 Bermuda_55 Bermuda_63 Bermuda_65 Bermuda_67 Bermuda_68 Bermuda_69 Bermuda_94 Bermuda_8 Bermuda_80 Bermuda_84 Bermuda_85 Bermuda_94 Guam_104460 Guam_104461 Guam_104464 Guam_104465 South_Africa 8_TX2017_028 82_TX2016_049 6_TX2017_027 59_TX2016_048 42_TX2016_046 13_TX2016_043 0.0 0.5 1.0

In [33]:
struct.result_files

['/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-0_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-1_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-2_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-3_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-4_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-5_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-6_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-7_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-8_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure/test-K-2-rep-9_f',
 '/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-struct

In [37]:
struct.workdir

'/gpfs/ysm/scratch60/dunn/rb956/finalproject/ipyrad09/analysis-structure'